In [ ]:
import os
import torch

# Set the base directory
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Define the Data directory path
data_dir = os.path.join(base_dir, "Data")

# PETS

# Import and inspect data

In [ ]:
import os
import pandas as pd

def load_annotations(annotation_file, image_dir):
    """
    Load annotations and create a DataFrame with image paths and labels.
    """
    # Load annotations file
    annotations = pd.read_csv(
        annotation_file, sep=" ", skiprows=6, header=None,
        names=["filename", "class_id", "species", "breed"]
    )
    # Add the full image path to each row
    annotations["image_path"] = annotations["filename"].apply(
        lambda x: os.path.join(image_dir, x + ".jpg")
    )
    return annotations

# Path to dataset
dataset_path = os.path.join(data_dir, "The Oxford-IIIT Pet Dataset")
image_dir = os.path.join(dataset_path, "images")
annotation_file = os.path.join(dataset_path, "annotations", "list.txt")

# Load the dataset
annotations = load_annotations(annotation_file, image_dir)

# Display the first few rows
print(annotations.head())

# Preprocessing

In [ ]:
from torchvision import transforms
from PIL import Image

def preprocess_image(image_path, transform):
    """
    Preprocess a single image by applying the given transformations.
    """
    img = Image.open(image_path).convert("RGB")  # Convert to RGB
    img_tensor = transform(img)  # Apply the transformations
    return img_tensor

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(           # Normalise using ImageNet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Feature Extraction

In [ ]:
import torch
import numpy as np
from torchvision.models import resnet18
from tqdm import tqdm

def extract_features(image_paths, model, transform, device):
    # Extract feature vectors for all images using the pre-trained model
    features = []
    for path in tqdm(image_paths, desc="Extracting Features"):
        img_tensor = preprocess_image(path, transform).unsqueeze(0).to(device)  # Move tensor to device
        with torch.no_grad():  # Disable gradient calculation
            feature = model(img_tensor).flatten().detach().cpu().numpy()  # Move to CPU and convert to NumPy
        features.append(feature)
    return features

# Load ResNet18 model and remove the classification layer
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") 
model = resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the final classification layer
model = model.to(device)
model.eval()

# Extract features for all images
image_paths = annotations["image_path"].tolist()
features = extract_features(image_paths, model, transform, device)

# Save the features
import numpy as np
np.save(os.path.join(data_dir, "pet_features.npy"), features)

# Dimensionality Reduction

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Load the saved features
features = np.load(os.path.join(data_dir, "pet_features.npy"))

# Reduce dimensions using t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
reduced_features = tsne.fit_transform(features)

# Save reduced features for future use
np.save(os.path.join(data_dir, "reduced_features.npy"), reduced_features)

In [ ]:
# Visualise the clusters
plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=annotations["class_id"], s=1)
plt.colorbar()
plt.title("t-SNE Visualisation of Pet Features")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.show()

# Clustering

In [ ]:
from sklearn.cluster import KMeans

# Perform k-Means clustering
num_clusters = 37  # Number of breeds in the dataset
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(reduced_features)

In [ ]:
# Visualise the clusters
plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=cluster_labels, s=1)
plt.colorbar()
plt.title("k-Means Clustering of Pet Features (t-SNE Reduced)")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.show()

## Davies-Bouldin Index (DBI) and Silhouette Score

In [ ]:
from sklearn.metrics import davies_bouldin_score, silhouette_score

# Calculate DBI
dbi = davies_bouldin_score(reduced_features, cluster_labels)
print(dbi)

# Calculate silhouette score
silhouette_avg = silhouette_score(reduced_features, cluster_labels)
print(silhouette_avg)

# Classification

## Simple logistic regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load extracted features and labels
features = np.load(os.path.join(data_dir, "pet_features.npy"))
labels = annotations["class_id"].values

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

# Train a Logistic Regression classifier
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the classifier
print(classification_report(y_test, y_pred, zero_division=0))

# Confusion Matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(conf_matrix)
plt.title("Simple Linear Regression Confusion Matrix for the Pets dataset")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

## SVM

In [ ]:
from sklearn.svm import SVC

# Train an SVM classifier
svm_clf = SVC(kernel='linear', random_state=42)
svm_clf.fit(X_train, y_train)

# Predict on the test set
y_pred_svm = svm_clf.predict(X_test)

# Evaluate SVM performance
print(classification_report(y_test, y_pred_svm, zero_division=0))

In [ ]:
# Confusion Matrix for SVM
conf_matrix_svm = confusion_matrix(y_test, y_pred_svm)
sns.heatmap(conf_matrix_svm)
plt.title("SVM Confusion Matrix for the Pets Dataset")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

## k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Train a k-NN classifier
knn_clf = KNeighborsClassifier(n_neighbors=5)  # Using 5 neighbours
knn_clf.fit(X_train, y_train)

# Predict on the test set
y_pred_knn = knn_clf.predict(X_test)

# Evaluate k-NN performance
print(classification_report(y_test, y_pred_knn, zero_division=0))

In [ ]:
# Confusion Matrix for k-NN
conf_matrix_knn = confusion_matrix(y_test, y_pred_knn)
sns.heatmap(conf_matrix_knn)
plt.title("k-NN Confusion Matrix for the Pets dataset")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

# Food-101

## importing the Food-101 Dataset

In [ ]:
import os
import pandas as pd

def load_food101_data(meta_folder, image_folder):
    """
    Load Food-101 dataset annotations and return a DataFrame with image paths and labels.
    """
    # Load train and test file paths
    train_file = os.path.join(meta_folder, "train.txt")
    test_file = os.path.join(meta_folder, "test.txt")

    # Read train and test files into dataframes
    train_df = pd.read_csv(train_file, header=None, names=["image_path"])
    test_df = pd.read_csv(test_file, header=None, names=["image_path"])

    # Extract labels from file paths
    train_df["label"] = train_df["image_path"].apply(lambda x: x.split("/")[0])
    test_df["label"] = test_df["image_path"].apply(lambda x: x.split("/")[0])

    # Add full image paths
    train_df["image_path"] = train_df["image_path"].apply(lambda x: os.path.join(image_folder, x + ".jpg"))
    test_df["image_path"] = test_df["image_path"].apply(lambda x: os.path.join(image_folder, x + ".jpg"))

    return train_df, test_df

# Define paths
dataset_path = os.path.join(data_dir, "Food 101 Dataset", "food-101")
meta_folder = os.path.join(dataset_path, "meta")
image_folder = os.path.join(dataset_path, "images")

# Load the dataset
train_df, test_df = load_food101_data(meta_folder, image_folder)

# Display sample rows
print(train_df.head())
print(test_df.head())

## Preprocessing

In [ ]:
from torchvision import transforms
from PIL import Image

def preprocess_image(image_path, transform):
    """
    Preprocess a single image by applying the specified transformations.
    """
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img)  
    return img_tensor

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),          # Convert image to PyTorch tensor
    transforms.Normalize(           # Normalise using ImageNet stats
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

## Feature Extraction

In [ ]:
import torch
from torchvision.models import resnet18
from tqdm import tqdm 

def extract_features(image_paths, model, transform, device):
    # Extract feature vectors for all images using the pre-trained model
    features = []
    for path in tqdm(image_paths, desc="Extracting Features"):
        img_tensor = preprocess_image(path, transform).unsqueeze(0).to(device)  # Move tensor to device
        with torch.no_grad():  # Disable gradient calculation
            feature = model(img_tensor).flatten().detach().cpu().numpy()  # Move to CPU and convert to NumPy
        features.append(feature)
    return features

# Load ResNet18 pre-trained model and remove the classification layer
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") 
model = resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove the final classification layer
model = model.to(device) 
model.eval()

# Extract features for training images
train_image_paths = train_df["image_path"].tolist()
train_features = extract_features(train_image_paths, model, transform, device)

# Save extracted features
np.save(os.path.join(data_dir, "food_train_features.npy"), train_features)

# Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# Load the extracted features
features = np.load(os.path.join(data_dir, "food_train_features.npy"))

# Apply PCA to reduce to 50 dimensions
pca = PCA(n_components=50, random_state=42)
pca_features = pca.fit_transform(features)
np.save(os.path.join(data_dir, "food_pca_features.npy"), pca_features) # Save PCA features

# Apply t-SNE to reduce to 2 dimensions
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
tsne_features = tsne.fit_transform(pca_features)

# Save reduced features for future use
np.save(os.path.join(data_dir, "food_tsne_features.npy"), tsne_features)

In [ ]:
# Visualise the t-SNE results
plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=train_df["label"].factorize()[0], s=1)
plt.colorbar()
plt.title("t-SNE Visualisation of Food-101 Features")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.show()

#  k-Means clustering 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score

pca_features = np.load(os.path.join(data_dir, "food_pca_features.npy"))

# Apply k-Means Clustering
n_clusters = 101  # One cluster per food
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(pca_features)

# Evaluate clustering quality
silhouette_avg = silhouette_score(pca_features, cluster_labels)
dbi = davies_bouldin_score(pca_features, cluster_labels)

print(silhouette_avg)
print(dbi)

In [ ]:
# Visualise clustering results
plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=cluster_labels, s=1)
plt.colorbar()
plt.title("k-Means Clustering of Food-101 Features (t-SNE Reduced)")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.show()

# Hierarchical Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch

def apply_hierarchical_clustering(features, n_clusters):

# Perform Agglomerative Hierarchical Clustering
    
    # Fit Agglomerative Clustering
    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    cluster_labels = clustering.fit_predict(features)

    # Evaluate clustering performance
    silhouette_avg = silhouette_score(features, cluster_labels)
    dbi = davies_bouldin_score(features, cluster_labels)

    print(silhouette_avg)
    print(dbi)

    return cluster_labels

# Visualise the Dendrogram
def plot_dendrogram(features, sample_size=500):
    
# Plotting a dendrogram using a subset of features.
    
    subset = features[:sample_size]  # Use the first 500 samples
    linkage_matrix = sch.linkage(subset, method='ward')

    sch.dendrogram(linkage_matrix)
    plt.title("Hierarchical Clustering Dendrogram")
    plt.xlabel("Sample Index")
    plt.ylabel("Distance")
    plt.show()

# Run Hierarchical Clustering
n_clusters = 101  # Number of food classes
plot_dendrogram(pca_features)  # Visualise dendrogram
hierarchical_labels = apply_hierarchical_clustering(pca_features, n_clusters=n_clusters)

In [ ]:
# Visualise Clustering
plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=hierarchical_labels, s=1)
plt.colorbar()
plt.title("Hierarchical Clustering of Food-101 Features (t-SNE Reduced)")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.show()

# DBSCAN

eps = 3 and min_samples = 10

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np
import matplotlib.pyplot as plt

def dbscan_clustering(reduced_features, eps=3.0, min_samples=10):

    # Apply DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
    cluster_labels = dbscan.fit_predict(reduced_features)

    # Evaluate clustering performance ignoring noise points
    valid_indices = cluster_labels != -1
    valid_labels = cluster_labels[valid_indices]
    valid_features = reduced_features[valid_indices]

    if len(np.unique(valid_labels)) > 1:  # Ensure valid clusters exist
        silhouette_avg = silhouette_score(valid_features, valid_labels)
        dbi = davies_bouldin_score(valid_features, valid_labels)
    else:
        silhouette_avg = None
        dbi = None

    # Print evaluation metrics
    print({silhouette_avg} if silhouette_avg else "Not enough clusters for Silhouette Score.")
    print({dbi} if dbi else "Not enough clusters for DBI.")

    return cluster_labels

# DBSCAN Parameters
eps = 3.0  # Distance threshold
min_samples = 10  # Minimum samples per cluster

# Run DBSCAN
dbscan_labels = dbscan_clustering(pca_features, eps=eps, min_samples=min_samples)

In [ ]:
# Visualise the results
plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=dbscan_labels, s=1)
plt.colorbar()
plt.title("DBSCAN Clustering of Food-101 Features (t-SNE Reduced)")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.show()

Increased eps:
Adjusted from 3.0 to 5.0 to consider a larger radius for neighbouring points.
Reduced min_samples:
Lowered from 10 to 5 to allow smaller clusters to form.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np
import matplotlib.pyplot as plt

def dbscan_clustering(reduced_features, eps=5.0, min_samples=5):

    # Apply DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
    cluster_labels = dbscan.fit_predict(reduced_features)

    # Evaluate clustering performance ignoring noise points
    valid_indices = cluster_labels != -1
    valid_labels = cluster_labels[valid_indices]
    valid_features = reduced_features[valid_indices]

    if len(np.unique(valid_labels)) > 1:  # Ensure valid clusters exist
        silhouette_avg = silhouette_score(valid_features, valid_labels)
        dbi = davies_bouldin_score(valid_features, valid_labels)
    else:
        silhouette_avg = None
        dbi = None

    # Print evaluation metrics
    print({silhouette_avg} if silhouette_avg else "Not enough clusters for Silhouette Score.")
    print({dbi} if dbi else "Not enough clusters for DBI.")

    return cluster_labels

# Adjusted DBSCAN Parameters
eps = 5.0  # Larger distance
min_samples = 5  # Lower minimum samples per cluster

# Run DBSCAN
dbscan_labels = dbscan_clustering(pca_features, eps=eps, min_samples=min_samples)

In [ ]:
# Visualise the results
plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=dbscan_labels, s=1)
plt.colorbar()
plt.title("DBSCAN Clustering of Food-101 Features (t-SNE Reduced)")
plt.xlabel("Dim 1")
plt.ylabel("Dim 2")
plt.show()

# Classification

## SVM

In [ ]:
# Extract features for testing images
test_image_paths = test_df["image_path"].tolist()

# Use the GPU if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

test_features = extract_features(test_image_paths, model, transform, device)

# Save the test features for reuse
np.save(os.path.join(data_dir, "food_test_features.npy"), test_features)

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the original features
train_features = np.load(os.path.join(data_dir, "food_train_features.npy"))
test_features = np.load(os.path.join(data_dir, "food_test_features.npy"))

# Ensure labels are correctly loaded
train_labels = train_df["label"].factorize()[0]
test_labels = test_df["label"].factorize()[0]

# Train SVM
svm = SVC(kernel="linear", C=1, random_state=42)
svm.fit(train_features, train_labels)

# Test SVM
predicted_labels = svm.predict(test_features)

# Generate Classification Report
print(classification_report(test_labels, predicted_labels))

In [ ]:
# Generate Confusion Matrix
plt.figure(figsize=(16, 12))
conf_matrix = confusion_matrix(test_labels, predicted_labels)
sns.heatmap(conf_matrix, xticklabels=np.unique(test_df["label"]),
            yticklabels=np.unique(test_df["label"]))
plt.title("SVM Confusion Matrix for Food-101")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

##  k-NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the features
train_features = np.load(os.path.join(data_dir, "food_train_features.npy"))
test_features = np.load(os.path.join(data_dir, "food_test_features.npy"))

# Load the labels
train_labels = train_df["label"].factorize()[0]
test_labels = test_df["label"].factorize()[0]

# Train k-NN
k = 5
knn = KNeighborsClassifier(n_neighbors=k, metric="euclidean")
knn.fit(train_features, train_labels)

# Test k-NN
predicted_labels = knn.predict(test_features)

# Generate classification report
print(classification_report(test_labels, predicted_labels))

# Generate confusion matrix
conf_matrix = confusion_matrix(test_labels, predicted_labels)

In [ ]:
# Visualise the confusion matrix
plt.figure(figsize=(16, 12))
sns.heatmap(conf_matrix,
            xticklabels=test_df["label"].unique(),
            yticklabels=test_df["label"].unique())
plt.title("k-NN Confusion Matrix for Food-101")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

## Fine tuning SVM

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV

# Load the features and labels
train_features = np.load(os.path.join(data_dir, "food_train_features.npy"))
test_features = np.load(os.path.join(data_dir, "food_test_features.npy"))
train_labels = train_df["label"].factorize()[0]
test_labels = test_df["label"].factorize()[0]

# Define parameter grid for GridSearchCV
param_grid = {
    "C": [0.1, 1, 10],  # Regularisation parameter
    "kernel": ["linear", "rbf"],  # Kernels to try
    "gamma": ["scale", "auto"],  # Kernel coefficient
}

# GridSearchCV for hyperparameter tuning
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=3, scoring="accuracy", verbose=2)
grid_search.fit(train_features, train_labels)

# Best hyperparameters
best_svm = grid_search.best_estimator_
print(f"Best Parameters: {grid_search.best_params_}")

# Evaluate the fine-tuned SVM
predicted_labels = best_svm.predict(test_features)

# Classification Report
print(classification_report(test_labels, predicted_labels, target_names=test_df["label"].unique()))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(16, 12))
conf_matrix = confusion_matrix(test_labels, predicted_labels)
sns.heatmap(conf_matrix, xticklabels=test_df["label"].unique(), yticklabels=test_df["label"].unique())
plt.title("Improved SVM Confusion Matrix for Food-101")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

In [ ]:
import joblib
joblib.dump(svm, os.path.join(data_dir, "food101_svm_model.pkl"))